In [1]:
# getting the dataset for training 
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-05-15 11:45:45--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2025-05-15 11:45:46 (7.91 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [2]:
# read and inspeect the text file
with open('input.txt', 'r') as f:
    text = f.read()

In [3]:
# exploring the txt file

# looking at first 1000 characters
print(text[:1000], '\n')

#number of characters in the dataset
print(f"length of the dataset: {len(text)}")

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
# vocabulary and the size of vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"vocabulary: {''.join(chars)}")
print(f"size of vocabulary: {vocab_size}")

vocabulary: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
size of vocabulary: 65


In [5]:
# tokenizing the text; ie convert the raw strings to some sequences of integers

# creating a mapping of characters to integers
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda st: [stoi[x] for x in st] # function of the string which takes the string and returns a list of integers
decode = lambda l: ''.join([itos[x] for x in l]) # function of the lost of integers where it takes each elements of the list and returns a character and joins them together

print(encode('hii there'))
print(decode(encode('hii there')))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [6]:
# encoding the entire dataset and storing it as a torch tensor
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [8]:
print(data[:100])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [9]:
# splitting the data into train and validation sets
n = int(0.9 * len(data)) # first 90% will be train, rest 10% will be validation set
train_data = data[:n]
val_data = data[n:]
# validation set will help us understand how much our model is overfitting and memorising 

In [16]:
# the entire text is never fed into the transformer in one go as it is computationally prohibitive
# chunking the dataset and sample these chunks into the transformer, thus training transformer on chunks at a time
block_size = 8 # chunk size
"""seeing first 9 chars
we are chunking for 8 chars. Since each preceding sequence of characters must predict for the next character. so, 18 would predict 47, 18 and 47 would predict 56.
So this way we have actually 8 individual training examples here
"""
train_data[:block_size + 1] 


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [19]:
x = train_data[: block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1] #context is all preceding chars plus current character
    target = y[t] 
    print(f" Example {t+1}: When input: {context}, target: {target}")

# this masked type attention is done to make transformer used to seeing all kinds of lengths of context

 Example 1: When input: tensor([18]), target: 47
 Example 2: When input: tensor([18, 47]), target: 56
 Example 3: When input: tensor([18, 47, 56]), target: 57
 Example 4: When input: tensor([18, 47, 56, 57]), target: 58
 Example 5: When input: tensor([18, 47, 56, 57, 58]), target: 1
 Example 6: When input: tensor([18, 47, 56, 57, 58,  1]), target: 15
 Example 7: When input: tensor([18, 47, 56, 57, 58,  1, 15]), target: 47
 Example 8: When input: tensor([18, 47, 56, 57, 58,  1, 15, 47]), target: 58


In [26]:
# we stack the batches of chunks into stacks and then feed them into the transformer. 
# we will have many batches of many chunks of text that are all stacked up in a single tensor.
# we do this because GPUs are very good at paralle processing.
# These chunks are trained upon independently

# we will start sampling random locations in the dataset to pull chunks from 

torch.manual_seed(1337) # we are setting the seed for the random number generator so that the random locations pulled are same whenever we train the model
batch_size = 4 # how many independent sequences will we process in parallel
block_size = 8 # what is the maximum context length for predictions

def get_batch(split):
    # generate a small batch of data of inputs x and target y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,)) # we generate batch_size number of random offsets, ie 4 values. 
    # these 4 values should be between 0 and len(data)-block_size
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    # torch.stack to take all 4 of the 1-D tensors and stack them up as rows in a 4x8 tensor
    return x, y

xb, yb = get_batch('train')
print(f'inputs: {xb.shape} shape, \n{xb}')
print(f'\ntargets: {yb.shape} shape, \n{yb}')
print('----------------')
for b in range(batch_size):
    for t in range (block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f'when input is {context.tolist()}, the target is {target}')

inputs: torch.Size([4, 8]) shape, 
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

targets: torch.Size([4, 8]) shape, 
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----------------
when input is [24], the target is 43
when input is [24, 43], the target is 58
when input is [24, 43, 58], the target is 5
when input is [24, 43, 58, 5], the target is 57
when input is [24, 43, 58, 5, 57], the target is 1
when input is [24, 43, 58, 5, 57, 1], the target is 46
when input is [24, 43, 58, 5, 57, 1, 46], the target is 43
when input is [24, 43, 58, 5, 57, 1, 46, 43], the target is 39
when input is [44], the target is 53
when input is [44, 53], the target is 56
when input is [44, 53, 56], the target is 1
when input is [44, 53, 56, 1], the target is 58
when 